In [1]:
# Download DHODH binding data from ChEMBL using the official python client from https://github.com/chembl/chembl_webresource_client
import itertools, os, os.path
import numpy as np, pandas as pd
import chembl_webresource_client.new_client

In [ ]:
from chembl_webresource_client.new_client import new_client

available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

In [ ]:
DHODH_chembl_id = 'CHEMBL1966' # (now solved - ) https://github.com/chembl/GLaDOS/issues/1319 

In [ ]:
df_activity = pd.DataFrame.from_records(chembl_webresource_client.new_client.new_client.activity.filter(target_chembl_id=DHODH_chembl_id, assay_type='B'))
print(len(df_activity))
df_activity.head(3).transpose()

In [ ]:
df_assay = pd.DataFrame.from_records(chembl_webresource_client.new_client.new_client.assay.filter(target_chembl_id=DHODH_chembl_id, assay_type='B'))
print(len(df_assay))
df_assay.head(3).transpose()

In [ ]:
df_actives = df_activity.dropna(subset=['pchembl_value']).copy()
df_actives['pchembl_value'] = pd.to_numeric(df_actives['pchembl_value'])
df_actives = df_actives.merge(df_assay[['assay_chembl_id', 'confidence_score']], on='assay_chembl_id').query('(confidence_score == 9)')
df_actives

In [ ]:
df_agg = df_actives.groupby(['canonical_smiles', 'molecule_chembl_id']).agg({
    'pchembl_value': np.mean,
    'molecule_pref_name': lambda x: next(iter(x)),
}).reset_index()
df_agg

In [ ]:
fp_ = os.path.join(os.path.abspath(""), 'DHODH_ChEMBL.tsv') #https://stackoverflow.com/questions/52119454/how-to-obtain-jupyter-notebooks-path
df_agg.to_csv(fp_, sep='\t', index=False, header=True)
!wc -l {fp_}

In [ ]:
fp_ = os.path.join(os.path.abspath(""), 'DHODH_ChEMBL.smi')
df_agg[['canonical_smiles', 'molecule_chembl_id']].to_csv(fp_, sep='\t', index=False, header=False)
!wc -l {fp_}